# Introduction

In this notebook I will be exploring the text structure of Reddit whisky reviews stored at 

# Imports/Setup

In [1]:
import os
import pandas as pd
import numpy as np
import pymongo
import spacy
from collections import Counter
from gensim.models import TfidfModel, LsiModel
from gensim.corpora import Dictionary

/usr/local/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
/usr/local/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


In [2]:
!sudo python -m spacy download en
nlp = spacy.load('en')

/usr/local/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
/usr/local/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
You are using pip version 18.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

    Linking successful
    /usr/local/anaconda/lib/python3.6/site-packages/en_core_web_sm -->
    /

/usr/local/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


# Data Load

# WhiskyReviews Database Connection

In [3]:
client = pymongo.MongoClient("mongodb://whiskyadmin:%s@whiskyreviews-shard-00-00-mvfds.mongodb.net:27017,whiskyreviews-shard-00-01-mvfds.mongodb.net:27017,whiskyreviews-shard-00-02-mvfds.mongodb.net:27017/test?ssl=true&replicaSet=WhiskyReviews-shard-0&authSource=admin&retryWrites=true" % (os.getenv('mongopw')))
reddit_reviews = client.reddit_reviews

In [4]:
reddit_reviews.list_collection_names()

['parsed', 'submissions']

In [5]:
parsed = reddit_reviews['parsed']

In [6]:
documents = [x for x in parsed.find()]

In [7]:
len(documents)

31126

In [8]:
documents[0]

{'_id': '54e266b4abda4e278055f0a3254b0ef2',
 'tokens': ['wife',
  'trip',
  'thailand',
  'meet',
  'family',
  'seen',
  'plenty',
  'whisky',
  'mostly',
  'jw',
  'one',
  'stood',
  'rest',
  '100',
  'pipers',
  'something',
  'seen',
  'seems',
  'quite',
  'following',
  'blend',
  '40',
  'alcohol',
  'volume',
  '35cl',
  '220',
  'baht',
  '$',
  '8',
  'canadian',
  'got',
  'novelty',
  'suspect',
  'thai',
  'equivalent',
  'chivas',
  'glenfiddich',
  '12',
  'colour',
  'caramel',
  'suspect',
  'artificially',
  'coloured',
  'nose',
  'tiger',
  'balm',
  'hands',
  'may',
  'way',
  'alcohol',
  'little',
  'bit',
  'leather',
  'hints',
  'sweetness',
  'palate',
  'bland',
  'taste',
  'almost',
  'nothing',
  'really',
  'bit',
  'woody',
  'flavour',
  'promise',
  'leather',
  'sweetness',
  'nose',
  'gone',
  'finish',
  'short',
  'devoid',
  'anything',
  'alcohol',
  'reminds',
  'jw',
  'red',
  'cheap',
  'rye',
  'dad',
  'drank',
  'kid',
  'bought',
  '

In [9]:
reviews = pd.DataFrame(documents)

In [10]:
reviews.head()

,_id,lemmas,tokens
0,54e266b4abda4e278055f0a3254b0ef2,"[wife, trip, thailand, meet, family, see, plen...","[wife, trip, thailand, meet, family, seen, ple..."
1,18b5c00c30a244a49bfa15132192417c,"[1792, series, continue, time, get, rare, stoc...","[1792, series, continues, time, getting, rare,..."
2,780c90af30b7447090fbc1fc1c857ecf,"[bourbon, review, 48, 1792, 225th, anniversary...","[bourbon, review, 48, 1792, 225th, anniversary..."
3,f86e6c62559743b4a20d85d0fb2594e8,"[1792, review, series, continue, bottle, bond,...","[1792, review, series, continues, bottled, bon..."
4,3bc513dc90d944439df8b2f6805b7edf,"[week, oak, liquor, cabinet, get, long, await,...","[week, oak, liquor, cabinet, got, long, awaite..."


In [11]:
all_words =  [item for sublist in reviews.tokens for item in sublist]

In [12]:
all_lemmas = [item for sublist in reviews.lemmas for item in sublist]

In [13]:
word_freq = Counter(all_words)

In [14]:
lemma_freq = Counter(all_lemmas)

In [15]:
word_freq.most_common(100)

[('finish', 72892),
 ('nose', 71168),
 ('100', 45640),
 ('oak', 44934),
 ('like', 44544),
 ('sweet', 42058),
 ('taste', 40747),
 ('whisky', 37718),
 ('one', 35166),
 ('vanilla', 35103),
 ('notes', 34646),
 ('bit', 32341),
 ('palate', 30576),
 ('really', 30461),
 ('good', 30380),
 ('abv', 29744),
 ('review', 29612),
 ('sherry', 28236),
 ('cask', 28086),
 ('bottle', 25912),
 ('peat', 25670),
 ('bourbon', 25046),
 ('would', 24691),
 ('smoke', 24674),
 ('malt', 23688),
 ('light', 23109),
 ('caramel', 22699),
 ('well', 22085),
 ('score', 21705),
 ('spice', 21527),
 ('honey', 21360),
 ('much', 21183),
 ('old', 21119),
 ('little', 21061),
 ('nice', 20404),
 ('|', 20233),
 ('time', 20009),
 ('year', 19998),
 ('fruit', 19973),
 ('quite', 19844),
 ('10', 19742),
 ('first', 18707),
 ('$', 18615),
 ('pepper', 17921),
 ('long', 17890),
 ('nbsp', 17256),
 ('12', 17255),
 ('rye', 17065),
 ('water', 16435),
 ('color', 16410),
 ('years', 16153),
 ('still', 15989),
 ('get', 15771),
 ('cinnamon', 15554),

In [16]:
lemma_freq.most_common(100)

[('finish', 79217),
 ('nose', 72261),
 ('taste', 53283),
 ('note', 48740),
 ('review', 47554),
 ('like', 47344),
 ('sweet', 45753),
 ('100', 45640),
 ('whisky', 45426),
 ('oak', 45374),
 ('good', 44084),
 ('bottle', 41716),
 ('cask', 39404),
 ('one', 36907),
 ('year', 36151),
 ('vanilla', 35149),
 ('bit', 32558),
 ('get', 31147),
 ('peat', 30988),
 ('palate', 30634),
 ('really', 30461),
 ('abv', 29744),
 ('spice', 29251),
 ('fruit', 29025),
 ('sherry', 28270),
 ('malt', 28259),
 ('smoke', 27339),
 ('bourbon', 27064),
 ('light', 25401),
 ('old', 25071),
 ('would', 24691),
 ('score', 23724),
 ('caramel', 22989),
 ('well', 22594),
 ('time', 22101),
 ('go', 22022),
 ('honey', 21791),
 ('dry', 21209),
 ('much', 21183),
 ('little', 21062),
 ('nice', 20766),
 ('lot', 20342),
 ('|', 20233),
 ('quite', 19844),
 ('10', 19765),
 ('apple', 19361),
 ('try', 18923),
 ('long', 18801),
 ('first', 18712),
 ('$', 18615),
 ('pepper', 18542),
 ('think', 18436),
 ('make', 18227),
 ('rye', 17900),
 ('water'

In [17]:
gensim_dict = Dictionary(reviews.lemmas)

In [18]:
print(gensim_dict)

Dictionary(41525 unique tokens: ['$', '100', '12', '220', '35cl']...)


In [19]:
gensim_dict.filter_extremes(no_above=0.75, no_below=20)

In [20]:
print(gensim_dict)

Dictionary(8368 unique tokens: ['$', '100', '12', '220', '35cl']...)


In [21]:
corpus = [gensim_dict.doc2bow(text) for text in reviews.lemmas]

In [22]:
model = TfidfModel(corpus)

In [23]:
reviews['tfidf_vector'] = [model[x] for x in corpus]

In [24]:
reviews.head(20)

,_id,lemmas,tokens,tfidf_vector
0,54e266b4abda4e278055f0a3254b0ef2,"[wife, trip, thailand, meet, family, see, plen...","[wife, trip, thailand, meet, family, seen, ple...","[(0, 0.03412816814276638), (1, 0.0175051348036..."
1,18b5c00c30a244a49bfa15132192417c,"[1792, series, continue, time, get, rare, stoc...","[1792, series, continues, time, getting, rare,...","[(7, 0.11693062811468903), (9, 0.0303607237337..."
2,780c90af30b7447090fbc1fc1c857ecf,"[bourbon, review, 48, 1792, 225th, anniversary...","[bourbon, review, 48, 1792, 225th, anniversary...","[(1, 0.007146252427824452), (13, 0.03518818716..."
3,f86e6c62559743b4a20d85d0fb2594e8,"[1792, review, series, continue, bottle, bond,...","[1792, review, series, continues, bottled, bon...","[(0, 0.02194097594289072), (1, 0.0056270195926..."
4,3bc513dc90d944439df8b2f6805b7edf,"[week, oak, liquor, cabinet, get, long, await,...","[week, oak, liquor, cabinet, got, long, awaite...","[(1, 0.004759590876690336), (7, 0.131373865310..."
5,26ba87ecb3d840de8e8231e8923aa334,"[1792, full, proof, price, approx, $, 45, abv,...","[1792, full, proof, price, approx, $, 45, abv,...","[(0, 0.03992361940043558), (13, 0.025208147869..."
6,ad782e77550743d69d77b766c0d8bae6,"[>, 69, 100, nice]","[>, 69, 100, nice]","[(1, 0.07238990168298674), (296, 0.24947556099..."
7,fd6aa09698c14a259e65e2ef4cc971e9,"[review, 9, 11, 1792, threesome, tap, review, ...","[reviews, 9, 11, 1792, threesome, tap, review,...","[(2, 0.015069043142827893), (8, 0.041280891136..."
8,18686c2d01eb42d1b71c8927becb6a77,"[regular, 1792, inoffensive, plain, bourbon, b...","[regular, 1792, inoffensive, plain, bourbon, b...","[(0, 0.040579288579111826), (1, 0.010407032599..."
9,efbe66ef04a44d73af78fc5c90b7ce19,"[pick, earlier, week, crack, right, away, dunn...","[picked, earlier, week, cracked, right, away, ...","[(0, 0.02274045708610662), (1, 0.0058320558713..."


In [25]:
gensim_lsi = LsiModel(corpus, id2word=gensim_dict, num_topics=100)

In [26]:
reviews['lsi_100'] = [gensim_lsi[x] for x in corpus]

In [27]:
reviews.head()

,_id,lemmas,tokens,tfidf_vector,lsi_100
0,54e266b4abda4e278055f0a3254b0ef2,"[wife, trip, thailand, meet, family, see, plen...","[wife, trip, thailand, meet, family, seen, ple...","[(0, 0.03412816814276638), (1, 0.0175051348036...","[(0, 3.33971128333082), (1, -0.284198568612503..."
1,18b5c00c30a244a49bfa15132192417c,"[1792, series, continue, time, get, rare, stoc...","[1792, series, continues, time, getting, rare,...","[(7, 0.11693062811468903), (9, 0.0303607237337...","[(0, 11.212749450229037), (1, 1.17891267756165..."
2,780c90af30b7447090fbc1fc1c857ecf,"[bourbon, review, 48, 1792, 225th, anniversary...","[bourbon, review, 48, 1792, 225th, anniversary...","[(1, 0.007146252427824452), (13, 0.03518818716...","[(0, 6.194591081511533), (1, 0.188943688210547..."
3,f86e6c62559743b4a20d85d0fb2594e8,"[1792, review, series, continue, bottle, bond,...","[1792, review, series, continues, bottled, bon...","[(0, 0.02194097594289072), (1, 0.0056270195926...","[(0, 11.30827134868932), (1, 1.084815162085012..."
4,3bc513dc90d944439df8b2f6805b7edf,"[week, oak, liquor, cabinet, get, long, await,...","[week, oak, liquor, cabinet, got, long, awaite...","[(1, 0.004759590876690336), (7, 0.131373865310...","[(0, 14.649387174162893), (1, 1.20498560381070..."


In [28]:
reviews.to_pickle(os.getenv('DOMINO_WORKING_DIR') + '/data/processed/reviews_vectorized.pkl')

## Save Gensim Assets

In [29]:
gensim_dict.save(os.getenv('DOMINO_WORKING_DIR') + '/models/gensim_lemma.dictionary')
model.save(os.getenv('DOMINO_WORKING_DIR') + '/models/lemma_tfidf.genmod')
gensim_lsi.save(os.getenv('DOMINO_WORKING_DIR') + '/models/lsi100_lemma.genmod')

## Add Fields to MongoDB

This code isn't working yet due to encoding of vectors being written into MongoDB.  I need to look into that and decide how to proceed.

In [30]:
#test case
row = reviews.loc[0]

In [31]:
errors = []

i=0

for doc in documents:
    id = doc['_id']
    lsi_vec = reviews[reviews._id == id].lsi_100
    tfidf_vec = reviews[reviews._id == id].tfidf_vector
    
    try:
        parsed.update({"_id": id}, {"$set": {"lsi_100": lis_vec, "tfidf_vec": tfidf_vec}})
    except:
        errors.append(id)
    
    
    if i%1000 == 0:
        print('Iteration Completed:', i)
        
        
    i+=1
    
    

Iteration Completed: 0
Iteration Completed: 1000
Iteration Completed: 2000
Iteration Completed: 3000
Iteration Completed: 4000
Iteration Completed: 5000
Iteration Completed: 6000
Iteration Completed: 7000
Iteration Completed: 8000
Iteration Completed: 9000
Iteration Completed: 10000
Iteration Completed: 11000
Iteration Completed: 12000
Iteration Completed: 13000
Iteration Completed: 14000
Iteration Completed: 15000
Iteration Completed: 16000
Iteration Completed: 17000
Iteration Completed: 18000
Iteration Completed: 19000
Iteration Completed: 20000
Iteration Completed: 21000
Iteration Completed: 22000
Iteration Completed: 23000
Iteration Completed: 24000
Iteration Completed: 25000
Iteration Completed: 26000
Iteration Completed: 27000
Iteration Completed: 28000
Iteration Completed: 29000
Iteration Completed: 30000
Iteration Completed: 31000
